In [ ]:
# ------ Importações gerais e Funções
import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import timedelta
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn import metrics

pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True, precision=4)

# ------ Funções

# Função para manipulação das datas
def separa_datas(df, col):
    temp = pd.DataFrame()
    print('\nResgatando informações do dataframe...')
    temp[col] = pd.to_datetime(df[col], format='%b-%Y')
#    dias = []  # cria lista vazia para ser preenchida com a informação dos dias
    mes_ano = []  # cria lista vazia para ser preenchida com mês/ano

    if temp[col].isnull().sum() > 0:
        print(f'\nATENÇÃO! Identificadas NaNs na feature {col}, impossível continuar.')
        print(f'Favor resolver as {temp[col].isnull().sum()} NaNs antes de tentar novamente. Seu bosta.')
    else:
        print('\nSeparando parâmetros de data e gerando nova série, só um instante por favor.')
        for i in range(0, len(temp)):  # preenche a lista com mês e ano
            mes_ano.append(temp.loc[i, col].strftime('%b-%Y').lower())
        print('\nTudo pronto.')

        return pd.Series(data=mes_ano, name='mes_ano')  # retorna uma série nova



# Função para buscar a explicação de cada feature no dicionário e analisar NaNs
def analise_geral(lista, dataframe: pd.DataFrame):
    """
    :param lista: lista de features a se verificar
    :dtype lista: list
    :param dataframe: o banco de dados que contem todas as features que serão verificadas
    :type dataframe: DataFrame
    """
    def significado(x: object):
        dic = pd.read_csv('./data/dictionary.csv', index_col=False)
        print(f'Nome: {str(x).upper()}')
        print(f'Definição: {dic[dic['Feature'] == str(x)]['Descrição'].values}')

    if len(lista) > 1:
        for i in range(0, len(lista)):
            print('\n')
            print(f'Feature #{i+1}/{len(lista)}')
            significado(lista[i])
            print(f'Tipo de dado: {str(dataframe[lista[i]].dtypes).upper()}')
            print(f'Quantidade total de NaNs: {dataframe[lista[i]].isnull().sum()}')
            print(f'Proporção desses NaNs: {round((dataframe[lista[i]].isnull().sum()/len(dataframe))*100, 2)}%')
    elif len(lista) == 1:
        print('\n')
        significado(lista[0])
        print(f'Tipo de dado: {str(dataframe[lista[0]].dtypes).upper()}')
        print(f'Quantidade total de NaNs: {dataframe[lista[0]].isnull().sum()}')
        print(f'Proporção desses NaNs: {round((dataframe[lista[0]].isnull().sum()/len(dataframe))*100, 2)}%')



# Função que analisa apenas as features sem NaNs
def analise_nans(lista, dataframe: pd.DataFrame):
    """
    :param lista: lista de features a se verificar
    :dtype lista: list
    :param dataframe: o banco de dados que contem todas as features que serão verificadas
    :type dataframe: DataFrame
    """
    def significado(x: object):
        dic = pd.read_csv('./data/dictionary.csv', index_col=False)
        print(f'Nome: {str(x).upper()}')
        print(f'Definição: {dic[dic['Feature'] == str(x)]['Descrição'].values}')
    print('\nVerificando...')

    if len(lista) > 1:
        if any(dataframe[lista].isnull().sum() > 1) == False:
            print('==> Sem NaNs nessa lista, parabéns.')
        else:
            print('Calculando tamanho da solicitação, só um instante por favor.')
            tam = len(dataframe[lista].isnull().sum()[dataframe[lista].isnull().sum() > 0])
            idx = 1
            for i in range(0, len(lista)):
                if dataframe[lista[i]].isnull().sum() == 0:
                    pass
                else:
                    print('\n')
                    print(f'Feature #{idx}/{tam}')
                    significado(lista[i])
                    print(f'Tipo de dado: {str(dataframe[lista[i]].dtypes).upper()}')
                    print(f'Quantidade total de NaNs: {dataframe[lista[i]].isnull().sum()}')
                    print(f'Proporção desses NaNs: {round((dataframe[lista[i]].isnull().sum()/len(dataframe))*100, 2)}%')
                    idx += 1
    elif len(lista) == 1:
        if dataframe[lista[0]].isnull().sum() == 0:
            print('==> Sem NaNs nessa lista, parabéns.')
        else:
            print('\n')
            significado(lista[0])
            print(f'Tipo de dado: {str(dataframe[lista[0]].dtypes).upper()}')
            print(f'Quantidade total de NaNs: {dataframe[lista[0]].isnull().sum()}')
            print(f'Proporção desses NaNs: {round((dataframe[lista[0]].isnull().sum()/len(dataframe))*100, 2)}%')



# Função para armazenar os valores de média e desvio padrão das features, para normalização dos valores
def parametros_zscore(df: pd.DataFrame, norm:list):
    """
    Args: df = DataFrame contendo os dados que serão utilizados
          norm = lista com a relação das features que precisam passar por normalização. Caso
                 norm=False a função utiliza as features do DataFrame original

    Returns: DataFrame contendo os valores de média e desvio padrão de cada feature; lista
            contendo o nome das features utilizadas
    """
    if norm:
        pass
    else:
    # Normalização apenas das colunas originais, sem contar as categóricas (como 'fez_hardship', por exemplo)
        norm = ['EFFR', 'expec6m', 'expec12m', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
            'term', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low',
            'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec',
            'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
            'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
            'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low',
            'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
            'mths_since_rcnt_il', 'total_rev_hi_lim', 'inq_fi', 'acc_open_past_24mths', 'avg_cur_bal',
            'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
            'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc','mths_since_recent_bc',
            'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats',
            'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
            'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
            'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort',
            'total_bc_limit', 'total_il_high_credit_limit', 'tempo_total_tomador', 'tempo_consulta',
            'orig_projected_additional_accrued_interest', 'deferral_term', 'hardship_amount', 'hardship_length',
            'hardship_dpd', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 'tempo_total_contrato',
            'tempo_payment_plan', 'annual_inc_joint', 'dti_joint', 'revol_bal_joint', 'sec_app_fico_range_low',
            'sec_app_fico_range_high', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc',
            'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths',
            'sec_app_collections_12_mths_ex_med']

    new_df = {}
    for name in norm:
        media = df[name].mean()
        desvio = df[name].std()
        new_df.update({name: {
                            'Media': media,
                            'DesvPadrao': desvio,
                            }
    })

    return pd.DataFrame(new_df), norm


# Função de ZScore personalizada
def tcc_zscore(df:pd.DataFrame, norm:list):
    from scipy import stats
    """
    Args: df = DataFrame contendo os dados para normalização
          norm = lista contendo o nome de cada feature para normalização
    """
    try:
        print('Iniciando processo de normalização.\n')
        for name in norm:
            df[name] = stats.zscore(df[name])
        print('Processo de normalização concluído com sucesso!')
    except Exception as e:
        print(f'O seguinte erro ocorreu durante a tentativa de padronização por ZScore: {e}. Favor verificar')


# Função de carregamento (parcial ou integral) do banco de dados
def carregamento_treino(percentual=1):
    print('Carregando banco de dados de treino completo...')
    try:
        modelagem = pd.read_parquet('data/modelagem.parquet')
    except Exception as e:
        print(f'Erro no carregamento: {e}. Favor verificar.')
        
    # O PC não conseguia processar o arquivo inteiro no modelo de LDA, optei por testar com apenas as 70% observações iniciais
    # O processamento com 70% produz resultado, com utilização de 100% CPU e 99% RAM durante o treino
    if (percentual < 1) and (percentual > 0):
        print('Realizando redução das observações.')
        treino_manter, treino_deletar = train_test_split(modelagem, train_size=percentual, random_state=1)
        print('Liberando memória.')
        del treino_deletar, modelagem

        print('Realizando separação variáveis dependente e independentes.')
        target = ['default']
        test_data = treino_manter[target]
        train_data = treino_manter.drop(target, axis = 1)
        del treino_manter
        print('Carregamento do banco de dados concluído.')
        
    elif percentual > 0:
        print('Banco de dados carregado com sucesso, separando variáveis dependente e independentes.')
        target = ['default']
        test_data = modelagem[target]
        train_data = modelagem.drop(target, axis = 1)
        del modelagem
        print('Carregamento do banco de dados concluído.')
    else:
        print('Favor registrar um percentual em decimais, entre 0 (exclusive) e 1 (inclusive).')
        test_data = None
        train_data = None
    
    return train_data, test_data


# Análise das métricas e geração da matriz de confusão
def analise_metricas(classif=None, X=pd.DataFrame, y=pd.Series):
    """
    Args:
    - classif: o algoritmo utilizado (o objeto que processa o .fit)
    - X: DataFrame (normalmente teste_x)
    - y: Series (normalmente teste_y)
    """
    if not classif:
        print('Necessário treinamento do modelo antes de se poder calcular as métricas')
    else:
        # ------ Cálculo das métricas do modelo
        print('Realizando a previsão dos valores do banco de dados e calculando as métricas de avaliação.\n')
        inicio_prev = timer()
        ypred = classif.predict(X)  # previsão dos dados de teste
        acc = accuracy_score(y, ypred)  # cálculo accuracy (eficiência geral do modelo)
        sens = recall_score(y, ypred, pos_label=1)  # cálculo da sensitividade/recall (taxa de acerto dos eventos)
        spec = recall_score(y, ypred, pos_label=0)  # cálculo da especificidade (taxa de acerto dos não-eventos)
        prec = precision_score(y, ypred)  # taxa de acerto dos positivos totais (TP/(TP+FP))
        fim_prev = timer()

        print(f'Análise do modelo:\n')
        print(f'Valor métrica Accuracy:: {acc:.6f}')
        print(f'Valor métrica Sensitividade (Recall): {sens:.6f}')
        print(f'Valor métrica Especificidade: {spec:.6f}')
        print(f'Valor métrica Precision: {prec:.6f}')
        print(f'Tempo de cálculo {timedelta(seconds=fim_prev-inicio_prev)}.')
        print('\n')

        # ------ Matriz de confusão
        conf_matrix = metrics.confusion_matrix(y.default.values.astype('int'), ypred, labels=[1, 0])
        plt.figure(figsize=(6, 6))
        sns.heatmap(conf_matrix,
                    fmt='d',
                    annot=True,
                    cmap='Oranges',
                    xticklabels=['Inadimplentes', 'Em dia'],
                    yticklabels=['Inadimplentes', 'Em dia'])
        plt.title(f'Matriz de Confusão', fontsize=12)
        plt.xlabel('Dados de teste', fontsize=12)
        plt.ylabel('Dados previstos', fontsize=12)
        plt.show()

In [ ]:

import tensorflow as tf
from tensorflow.keras.losses import categorical_hinge


try:
    print('Iniciando timer geral e carregando banco de dados de treino.')
    inicio_geral = timer()
    train_data, test_data = carregamento_treino(0.7)
    
    print('Calculando dados de média e desvio padrão antes da padronização no banco de dados. Também registrando o nome das features utilizadas')
    historico, norm = parametros_zscore(train_data, norm=False)
    print('Informações calculadas, histórico mantido no objeto "historico".\n')
    tcc_zscore(train_data, norm)
    print('Normalização do banco de dados realizada, prosseguindo.\n')

       
    # ------ Separação dados de treino e teste
    print('Realizando procedimentos de separação de amostras para treino do modelo.')
    target = ['default']
    tamanho_treino = 0.8
    treino_x, teste_x, treino_y, teste_y = train_test_split(train_data, test_data, train_size=tamanho_treino, random_state=1)
    print('Liberando memória...')
    del train_data, test_data  # libera memória
    print('Pronto.\n')


    # ------ Parâmetros do modelo
    print('Estabelecendo parâmetros do algoritmo de treino.')
    nome_modelo = datetime.datetime.now().strftime("%Y%m%d-%H%M")
    classif_nn = tf.keras.Sequential()
    #modelo_dnn = keras.Sequential([normalizador])
    classif_nn.add(tf.keras.layers.Dense(64, activation='relu', name='Input_Dense64'))
    classif_nn.add(tf.keras.layers.Dropout(0.3, seed=1))
    classif_nn.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden1_Dense64'))
    #classif_nn.add(tf.keras.layers.Dense(128, activation='relu', name='HiddenL3_Dense128'))
    classif_nn.add(tf.keras.layers.Dense(1, name='Output_Dense1'))
    classif_nn.compile(loss=categorical_hinge, 
                    optimizer='Adam',
                    metrics=['categorical_hinge'])

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',  # callback de early stopping
                                            patience=3)



    classif_nn.fit(treino_x,
                            treino_y,
                            epochs=3,
                            validation_data=(teste_x, teste_y),
                            batch_size=32,
                            callbacks=[callback])

except Exception as e:
    print(f'Erro durante o processo de treinamento: {e}! Favor verificar.')